In [1]:
import logging
import os
import pickle
import traceback
from itertools import product

from cods.od.cp import ODConformalizer
from cods.od.data import MSCOCODataset
from cods.od.models import DETRModel

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = (
    "0"  # chose the GPU. If only one, then "0"
)

logging.getLogger().setLevel(logging.DEBUG)

# set [COCO_PATH] to the directory to your local copy of the COCO dataset
COCO_PATH = "/datasets/shared_datasets/coco/"

data = MSCOCODataset(root=COCO_PATH, split="val")

calibration_ratio = (
    0.5  # set 0.5 to use 50% for calibration and 50% for testing
)

use_smaller_subset = True  # TODO: Temp

if use_smaller_subset:
    data_cal, data_val = data.split_dataset(
        calibration_ratio, shuffle=False, n_calib_test=800
    )
else:
    data_cal, data_val = data.split_dataset(calibration_ratio, shuffle=True)

# model and weights are downloaded from https://github.com/facebookresearch/detr
model = DETRModel(model_name="detr_resnet50", pretrained=True, device="cpu")
# model = YOLOModel(model_name="yolov8x.pt", pretrained=True)


print(f"{len(data) = }")
print(f"{len(data_cal) = }")
print(f"{len(data_val) = }")

preds_cal = model.build_predictions(
    data_cal,
    dataset_name="mscoco",
    split_name="cal",
    batch_size=12,
    collate_fn=data._collate_fn,  # TODO: make this a default for COCO
    shuffle=False,
    force_recompute=False,  # False,
    deletion_method="nms",
    filter_preds_by_confidence=1e-3,
)
preds_val = model.build_predictions(
    data_val,
    dataset_name="mscoco",
    split_name="test",
    batch_size=12,
    collate_fn=data._collate_fn,
    shuffle=False,
    force_recompute=False,  # False,
    deletion_method="nms",
    filter_preds_by_confidence=1e-3,
)

results = {}

alphas = [[0.03, 0.1, 0.1]]
matching_functions = ["mix"]  # , "hausdorff", "lac"]#["giou", "hausdorff"]
confidence_methods = [
    "box_count_threshold",
    # "box_count_recall",
    # "box_thresholded_distance",
]
localization_methods = ["pixelwise"]  # , "boxwise"]
classification_prediction_sets = ["aps"]
localization_prediction_sets = ["additive"]  # , "multiplicative"]

configs = []
for alpha in alphas:
    for matching_function in matching_functions:
        for confidence_method in confidence_methods:
            for localization_method in localization_methods:
                for (
                    classification_prediction_set
                ) in classification_prediction_sets:
                    for (
                        localization_prediction_set
                    ) in localization_prediction_sets:
                        configs.append(
                            {
                                "alpha": alpha,
                                "matching_function": matching_function,
                                "confidence_method": confidence_method,
                                "localization_method": localization_method,
                                "classification_prediction_set": classification_prediction_set,
                                "localization_prediction_set": localization_prediction_set,
                            }
                        )
for config in configs:
    try:
        conf = ODConformalizer(
            guarantee_level="image",
            matching_function=config["matching_function"],
            multiple_testing_correction=None,
            # confidence_method=config["confidence_method"],
            confidence_threshold=0.1,
            localization_method=config["localization_method"],
            localization_prediction_set=config["localization_prediction_set"],
            classification_method="binary",
            classification_prediction_set=config[
                "classification_prediction_set"
            ],
            backend="auto",
            optimizer="binary_search",
        )

        parameters = conf.calibrate(
            preds_cal,
            alpha_confidence=config["alpha"][0],
            alpha_localization=config["alpha"][1],
            alpha_classification=config["alpha"][2],
        )

        conformal_preds_cal = conf.conformalize(
            preds_cal, parameters=parameters
        )

        results_cal = conf.evaluate(
            preds_cal,
            parameters=parameters,
            conformalized_predictions=conformal_preds_cal,
            include_confidence_in_global=False,
        )

        conformal_preds = conf.conformalize(preds_val, parameters=parameters)

        results_val = conf.evaluate(
            preds_val,
            parameters=parameters,
            conformalized_predictions=conformal_preds,
            include_confidence_in_global=False,
        )

        config_str = f"alpha-{config['alpha']}-{config['matching_function']}_{config['confidence_method']}_{config['localization_method']}_{config['classification_prediction_set']}_{config['localization_prediction_set']}"

        results[config_str] = results_val

        print(f"Results for config {config_str}:")
        print(f"  {results_val}")
        # Save results to a pickle file
    except Exception as e:
        print(f"Error with config {config}: {e}")
        print(traceback.format_exc())
        continue


[2025-04-15 14:58:52:INFO:models.py:models:30 - __init__ ] Model detr_resnet50 initialized
Using cache found in /home/leo.andeol/.cache/torch/hub/facebookresearch_detr_main
/home/leo.andeol/envs/cods_13/cods/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/leo.andeol/envs/cods_13/cods/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
[2025-04-15 14:58:53:INFO:models.py:models:76 - _load_preds_if_exists ] Loading predictions from ./saved_predictions/2c92d1aaa0cc2db665dc992cc2c

len(data) = 5000
len(data_cal) = 400
len(data_val) = 400
Predictions already exist, loading them...
Predictions already exist, loading them...


[2025-04-15 14:58:53:WARNING:cp.py:cp:1488 - __init__ ] No multiple_testing_correction provided, assuming no correction is needed. The explicit list of alphas is expected for calibration.
[2025-04-15 14:58:53:INFO:cp.py:cp:192 - __init__ ] Defaulting to CRC backend
[2025-04-15 14:58:53:INFO:cp.py:cp:1031 - __init__ ] Defaulting to CRC backend
[2025-04-15 14:58:53:INFO:cp.py:cp:1764 - calibrate ] Matching Predictions to True Boxes
400it [00:00, 8450.47it/s]
[2025-04-15 14:58:53:INFO:cp.py:cp:1777 - calibrate ] Calibrating Localization Conformalizer
[2025-04-15 14:58:53:INFO:cp.py:cp:464 - calibrate ] Using overload confidence threshold: 0.1000
[1.71, 1.95] -> 1.8310546875. Corrected Risk = 0.10: 100%|██████████| 13/13 [00:49<00:00,  3.78s/it]
[2025-04-15 14:59:42:INFO:cp.py:cp:521 - calibrate ] Calibrated λ for localization: 1.8310546875
[2025-04-15 14:59:42:INFO:cp.py:cp:1787 - calibrate ] Calibrated Localization λ : 1.8310546875
[2025-04-15 14:59:42:INFO:cp.py:cp:1795 - calibrate ] Ca

Confidence threshold is 0.1
Matching is : True
Matching complete
Confidence threshold: 0.1
ODParameters
global_alpha: 0.23
alpha_confidence: 0.03
alpha_localization: 0.1
alpha_classification: 0.1
lambda_confidence_plus: None
lambda_confidence_minus: None
lambda_localization: 1.8310546875
lambda_classification: 0.32293620705604553
confidence_threshold: 0.1


[2025-04-15 15:01:54:INFO:cp.py:cp:2015 - evaluate ] Evaluation Results:
[2025-04-15 15:01:54:INFO:cp.py:cp:2023 - evaluate ] 	 Localization:
[2025-04-15 15:01:54:INFO:cp.py:cp:2024 - evaluate ] 		 Risk: 0.09
[2025-04-15 15:01:54:INFO:cp.py:cp:2025 - evaluate ] 		 Mean Set Size: 1.08
[2025-04-15 15:01:54:INFO:cp.py:cp:2029 - evaluate ] 	 Classification:
[2025-04-15 15:01:54:INFO:cp.py:cp:2030 - evaluate ] 		 Risk: 0.14
[2025-04-15 15:01:54:INFO:cp.py:cp:2031 - evaluate ] 		 Mean Set Size: 0.64
[2025-04-15 15:01:54:INFO:cp.py:cp:2035 - evaluate ] 	 Global:
[2025-04-15 15:01:54:INFO:cp.py:cp:2039 - evaluate ] 		 Risk: 0.18673092126846313
[2025-04-15 15:01:54:INFO:cp.py:cp:1847 - conformalize ] Conformalizing Predictions
[2025-04-15 15:01:54:INFO:cp.py:cp:1850 - conformalize ] Using provided parameters for conformalization
[2025-04-15 15:01:54:INFO:cp.py:cp:1856 - conformalize ] The parameters have been computed on another set of predictions.
[2025-04-15 15:01:54:INFO:cp.py:cp:1876 - conf

Confidence threshold is 0.1
Matching is : True
Matching complete
Confidence threshold: 0.1
ODParameters
global_alpha: 0.23
alpha_confidence: 0.03
alpha_localization: 0.1
alpha_classification: 0.1
lambda_confidence_plus: None
lambda_confidence_minus: None
lambda_localization: 1.8310546875
lambda_classification: 0.32293620705604553
confidence_threshold: 0.1


[2025-04-15 15:01:54:INFO:cp.py:cp:2015 - evaluate ] Evaluation Results:
[2025-04-15 15:01:54:INFO:cp.py:cp:2023 - evaluate ] 	 Localization:
[2025-04-15 15:01:54:INFO:cp.py:cp:2024 - evaluate ] 		 Risk: 0.12
[2025-04-15 15:01:54:INFO:cp.py:cp:2025 - evaluate ] 		 Mean Set Size: 1.09
[2025-04-15 15:01:54:INFO:cp.py:cp:2029 - evaluate ] 	 Classification:
[2025-04-15 15:01:54:INFO:cp.py:cp:2030 - evaluate ] 		 Risk: 0.15
[2025-04-15 15:01:54:INFO:cp.py:cp:2031 - evaluate ] 		 Mean Set Size: 0.64
[2025-04-15 15:01:54:INFO:cp.py:cp:2035 - evaluate ] 	 Global:
[2025-04-15 15:01:54:INFO:cp.py:cp:2039 - evaluate ] 		 Risk: 0.2156369984149933


Results for config alpha-[0.03, 0.1, 0.1]-mix_box_count_threshold_pixelwise_aps_additive:


In [1]:
import logging
import os
import pickle
import traceback
from itertools import product

from cods.od.cp import ODConformalizer
from cods.od.data import MSCOCODataset
from cods.od.models import DETRModel

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = (
    "0"  # chose the GPU. If only one, then "0"
)

logging.getLogger().setLevel(logging.DEBUG)

# set [COCO_PATH] to the directory to your local copy of the COCO dataset
COCO_PATH = "/datasets/shared_datasets/coco/"

data = MSCOCODataset(root=COCO_PATH, split="val")

calibration_ratio = (
    0.5  # set 0.5 to use 50% for calibration and 50% for testing
)

use_smaller_subset = True  # TODO: Temp

if use_smaller_subset:
    data_cal, data_val = data.split_dataset(
        calibration_ratio, shuffle=False, n_calib_test=800
    )
else:
    data_cal, data_val = data.split_dataset(calibration_ratio, shuffle=True)

# model and weights are downloaded from https://github.com/facebookresearch/detr
model = DETRModel(model_name="detr_resnet50", pretrained=True, device="cpu")
# model = YOLOModel(model_name="yolov8x.pt", pretrained=True)


print(f"{len(data) = }")
print(f"{len(data_cal) = }")
print(f"{len(data_val) = }")

preds_cal = model.build_predictions(
    data_cal,
    dataset_name="mscoco",
    split_name="cal",
    batch_size=12,
    collate_fn=data._collate_fn,  # TODO: make this a default for COCO
    shuffle=False,
    force_recompute=False,  # False,
    deletion_method="nms",
    filter_preds_by_confidence=1e-3,
)
preds_val = model.build_predictions(
    data_val,
    dataset_name="mscoco",
    split_name="test",
    batch_size=12,
    collate_fn=data._collate_fn,
    shuffle=False,
    force_recompute=False,  # False,
    deletion_method="nms",
    filter_preds_by_confidence=1e-3,
)

results = {}

alphas = [[0.03, 0.1, 0.1]]
matching_functions = ["mix"]  # , "hausdorff", "lac"]#["giou", "hausdorff"]
confidence_methods = [
    "box_count_threshold",
    # "box_count_recall",
    # "box_thresholded_distance",
]
localization_methods = ["pixelwise"]  # , "boxwise"]
classification_prediction_sets = ["aps"]
localization_prediction_sets = ["additive"]  # , "multiplicative"]

configs = []
for alpha in alphas:
    for matching_function in matching_functions:
        for confidence_method in confidence_methods:
            for localization_method in localization_methods:
                for (
                    classification_prediction_set
                ) in classification_prediction_sets:
                    for (
                        localization_prediction_set
                    ) in localization_prediction_sets:
                        configs.append(
                            {
                                "alpha": alpha,
                                "matching_function": matching_function,
                                "confidence_method": confidence_method,
                                "localization_method": localization_method,
                                "classification_prediction_set": classification_prediction_set,
                                "localization_prediction_set": localization_prediction_set,
                            }
                        )
for config in configs:
    try:
        conf = ODConformalizer(
            guarantee_level="image",
            matching_function=config["matching_function"],
            multiple_testing_correction=None,
            confidence_method=config["confidence_method"],
            # confidence_threshold=0.1,
            localization_method=config["localization_method"],
            localization_prediction_set=config["localization_prediction_set"],
            classification_method="binary",
            classification_prediction_set=config[
                "classification_prediction_set"
            ],
            backend="auto",
            optimizer="binary_search",
        )

        parameters = conf.calibrate(
            preds_cal,
            alpha_confidence=config["alpha"][0],
            alpha_localization=config["alpha"][1],
            alpha_classification=config["alpha"][2],
        )

        conformal_preds_cal = conf.conformalize(
            preds_cal, parameters=parameters
        )

        results_cal = conf.evaluate(
            preds_cal,
            parameters=parameters,
            conformalized_predictions=conformal_preds_cal,
            include_confidence_in_global=False,
        )

        conformal_preds = conf.conformalize(preds_val, parameters=parameters)

        results_val = conf.evaluate(
            preds_val,
            parameters=parameters,
            conformalized_predictions=conformal_preds,
            include_confidence_in_global=False,
        )

        config_str = f"alpha-{config['alpha']}-{config['matching_function']}_{config['confidence_method']}_{config['localization_method']}_{config['classification_prediction_set']}_{config['localization_prediction_set']}"

        results[config_str] = results_val

        print(f"Results for config {config_str}:")
        print(f"  {results_val}")
        # Save results to a pickle file
    except Exception as e:
        print(f"Error with config {config}: {e}")
        print(traceback.format_exc())
        continue


[2025-04-17 03:32:57:INFO:models.py:models:30 - __init__ ] Model detr_resnet50 initialized
Using cache found in /home/leo.andeol/.cache/torch/hub/facebookresearch_detr_main
/home/leo.andeol/envs/cods_13/cods/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/leo.andeol/envs/cods_13/cods/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
[2025-04-17 03:33:01:INFO:models.py:models:76 - _load_preds_if_exists ] Loading predictions from ./saved_predictions/2c92d1aaa0cc2db665dc992cc2c

len(data) = 5000
len(data_cal) = 400
len(data_val) = 400
Predictions already exist, loading them...


[2025-04-17 03:33:01:WARNING:cp.py:cp:1489 - __init__ ] No multiple_testing_correction provided, assuming no correction is needed. The explicit list of alphas is expected for calibration.
[2025-04-17 03:33:01:INFO:cp.py:cp:192 - __init__ ] Defaulting to CRC backend
[2025-04-17 03:33:01:INFO:cp.py:cp:1031 - __init__ ] Defaulting to CRC backend
[2025-04-17 03:33:01:INFO:cp.py:cp:1729 - calibrate ] Calibrating Confidence Conformalizer
[2025-04-17 03:33:01:DEBUG:cp.py:cp:822 - calibrate ] Optimizing for lambda_plus


Predictions already exist, loading them...


  0%|          | 0/400 [00:00<?, ?it/s]/home/leo.andeol/envs/cods_13/cods/cods/od/utils.py:424: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  Qst = torch.FloatTensor([Qs]).to(device)
100%|██████████| 400/400 [00:00<00:00, 1451.47it/s]
[2025-04-17 03:33:01:DEBUG:optim.py:optim:205 - optimize ] Risk after 1st epoch is 0.004987531341612339 < 0.03


...

Confidence threshold is 0.013575732707977295
Matching is : True
Matching complete
Confidence threshold: 0.013575732707977295
ODParameters
global_alpha: 0.23
alpha_confidence: 0.03
alpha_localization: 0.1
alpha_classification: 0.1
lambda_confidence_plus: 0.9864242672920227
lambda_confidence_minus: 0.9846822023391724
lambda_localization: 1.3427734375
lambda_classification: 0.906660407781601
confidence_threshold: 0.013575732707977295


[2025-04-17 03:34:39:INFO:cp.py:cp:2016 - evaluate ] Evaluation Results:
[2025-04-17 03:34:39:INFO:cp.py:cp:2018 - evaluate ] 	 Confidence:
[2025-04-17 03:34:39:INFO:cp.py:cp:2019 - evaluate ] 		 Risk: 0.03
[2025-04-17 03:34:39:INFO:cp.py:cp:2020 - evaluate ] 		 Mean Set Size: 25.06
[2025-04-17 03:34:39:INFO:cp.py:cp:2024 - evaluate ] 	 Localization:
[2025-04-17 03:34:39:INFO:cp.py:cp:2025 - evaluate ] 		 Risk: 0.10
[2025-04-17 03:34:39:INFO:cp.py:cp:2026 - evaluate ] 		 Mean Set Size: 1.07
[2025-04-17 03:34:39:INFO:cp.py:cp:2030 - evaluate ] 	 Classification:
[2025-04-17 03:34:39:INFO:cp.py:cp:2031 - evaluate ] 		 Risk: 0.10
[2025-04-17 03:34:39:INFO:cp.py:cp:2032 - evaluate ] 		 Mean Set Size: 0.70
[2025-04-17 03:34:39:INFO:cp.py:cp:2036 - evaluate ] 	 Global:
[2025-04-17 03:34:39:INFO:cp.py:cp:2040 - evaluate ] 		 Risk: 0.1638501137495041
[2025-04-17 03:34:39:INFO:cp.py:cp:1848 - conformalize ] Conformalizing Predictions
[2025-04-17 03:34:39:INFO:cp.py:cp:1851 - conformalize ] Using

Confidence threshold is 0.013575732707977295
Matching is : True
Matching complete
Confidence threshold: 0.013575732707977295
ODParameters
global_alpha: 0.23
alpha_confidence: 0.03
alpha_localization: 0.1
alpha_classification: 0.1
lambda_confidence_plus: 0.9864242672920227
lambda_confidence_minus: 0.9846822023391724
lambda_localization: 1.3427734375
lambda_classification: 0.906660407781601
confidence_threshold: 0.013575732707977295


[2025-04-17 03:34:40:INFO:cp.py:cp:2016 - evaluate ] Evaluation Results:
[2025-04-17 03:34:40:INFO:cp.py:cp:2018 - evaluate ] 	 Confidence:
[2025-04-17 03:34:40:INFO:cp.py:cp:2019 - evaluate ] 		 Risk: 0.03
[2025-04-17 03:34:40:INFO:cp.py:cp:2020 - evaluate ] 		 Mean Set Size: 28.26
[2025-04-17 03:34:40:INFO:cp.py:cp:2024 - evaluate ] 	 Localization:
[2025-04-17 03:34:40:INFO:cp.py:cp:2025 - evaluate ] 		 Risk: 0.12
[2025-04-17 03:34:40:INFO:cp.py:cp:2026 - evaluate ] 		 Mean Set Size: 1.08
[2025-04-17 03:34:40:INFO:cp.py:cp:2030 - evaluate ] 	 Classification:
[2025-04-17 03:34:40:INFO:cp.py:cp:2031 - evaluate ] 		 Risk: 0.10
[2025-04-17 03:34:40:INFO:cp.py:cp:2032 - evaluate ] 		 Mean Set Size: 0.71
[2025-04-17 03:34:40:INFO:cp.py:cp:2036 - evaluate ] 	 Global:
[2025-04-17 03:34:40:INFO:cp.py:cp:2040 - evaluate ] 		 Risk: 0.18737010657787323


Results for config alpha-[0.03, 0.1, 0.1]-mix_box_count_threshold_pixelwise_aps_additive:
